In [2]:
# Import the raw data from Live Coin Watch and initialize the dataframe.


In [3]:
import re # Regular expressions
import pandas as pd # Dataframe module
import numpy as np # Math module
from datetime import datetime # Date conversions 

with open("data/test_data.txt","r") as data_file: # "with" closes the file automatically.
    lines = data_file.readlines()
    
data_headers = ['date', 'rate', 'volume', 'cap']

df = pd.DataFrame(columns=data_headers)  

In [4]:
# Extract the relevant information from the raw data and store in the dataframe.
# Note that 'rate' refers to price. For example, 42306 would be $423.06

In [5]:
count = 0
for line in lines:
    count+= 1
    line_trunc = " ".join(line.strip().split()[1:3])
    indicator = re.compile('\d+:.*.*date:')
    result = indicator.search(line_trunc)
    if result != None:
        
        # Extract date, rate, volume, cap from this line of data.
        row = []
        for header in data_headers:
            header_pattern = re.compile('%s:.\d+' % header)
            try:
                row.append(header_pattern.search(line).group())
            except AttributeError:
                print("Something's gone wrong with pattern matching the date, rate, volume, and cap...")
                print("Data line %s content: %s" % (count, line))
        row = [x.split()[1] for x in row]
        row = pd.Series(row, index = df.columns)
        df = df.append(row, ignore_index = True)

In [6]:
# Convert unix timestamps in milliseconds to a human readable format and store in a new column.

In [7]:
df['date readable'] = [datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S') for x in df['date']]

In [8]:
# Calculate rates of change for each point.

In [42]:
# Calculates rates of change per 24 hour period
#     using values averaged over the given window
# Data must be a pandas dataframe with "rate" and "date" columns.
# Window must be an integer.
def calculate_rate_of_change(data, window):
    deltas = []
    index = 0
    for x in data['rate']:
        if index >= len(data)-window:
            pass
        else:
            current_average = 0
            next_average = 0
            for y in range(0, window):
                current_average += int(data['rate'][y])
                next_next_average = int(data['rate'][window + y])

            current_average = current_average / window
            next_average = next_average / window
            
            current_date = int(df['date'][index])
            next_date = int(df['date'][index+1])
            
            time_diff_in_days = (next_date - current_date) / 86400000 # assumes unix timestamps are in milliseconds
            deltas.append((next_average - current_average) / time_diff_in_days)
            index += 1
    for i in range(0, window):
        deltas.append(deltas[len(data)-window-1])
    return deltas

deltas = calculate_rate_of_change(df, 1)


df['rates of change'] = deltas
            

302
302


In [43]:
df.head()

,date,rate,volume,cap,date readable,rates of change
0,1642106400000,42606,15884015374,806469865583,2022-01-13 20:40:00,-142680.55814
1,1642132200000,42700,34958610821,808262490748,2022-01-14 03:50:00,-142680.55814
2,1642158000000,42134,27334118224,797560503949,2022-01-14 11:00:00,-142680.55814
3,1642183800000,42957,20222197339,813139819925,2022-01-14 18:10:00,-142680.55814
4,1642209600000,43150,15015629680,816812254015,2022-01-15 01:20:00,-142680.55814
